# Yoga Recommender: Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain 

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

### Context
Previously we saw that the model told us how to to change the tire, however we had to manually provide it with the relevant data and provide the contex ourselves. We explored the approach to leverage the model availabe under Bedrock and ask questions based on it's knowledge learned during training as well as providing manual context. While that approach works with short documents or single-ton applications, it fails to scale to enterprise level question answering where there could be large enterprise documents which cannot all be fit into the prompt sent to the model. 

### Pattern
We can improve upon this process by implementing an architecure called Retreival Augmented Generation (RAG). RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. 

In this notebook we explain how to approach the pattern of Question Answering to find and leverage the documents to provide answers to the user questions.

### Challenges
- How to manage large document(s) that exceed the token limit
- How to find the document(s) relevant to the question being asked

### Proposal
To the above challenges, this notebook proposes the following strategy
#### Prepare documents
![Embeddings](./images/Embeddings_lang.png)

Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Load the documents
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings
#### Ask question
![Question](./images/Chatbot_lang.png)

When the documents index is prepared, you are ready to ask the questions and relevant documents will be fetched based on the question being asked. Following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

## Usecase
#### Dataset
To explain this architecture pattern we are using the documents from IRS. These documents explain topics such as:
- Original Issue Discount (OID) Instruments
- Reporting Cash Payments of Over $10,000 to IRS
- Employer's Tax Guide

#### Persona
Let's assume a persona of a layman who doesn't have an understanding of how IRS works and if some actions have implications or not.

The model will try to answer from the documents in easy language.


## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V1 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: FAISS available through LangChain

  In this notebook we are using this in-memory vector-store to store both the embeddings and the documents. In an enterprise context this could be replaced with a persistent store such as AWS OpenSearch, RDS Postgres with pgVector, ChromaDB, Pinecone or Weaviate.
- **Index**: VectorIndex

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In this notebook, we'll also need some extra dependencies:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files

In [2]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.249 "pypdf>=3.8,<4"

Processing /root/dependencies/awscli-1.29.21-py3-none-any.whl
Processing /root/dependencies/boto3-1.28.21-py3-none-any.whl
Processing /root/dependencies/botocore-1.31.21-py3-none-any.whl
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Obtaining dependency information for s3transfer<0.7.0,>=0.6.0 from https://files.pythonhosted.org/packages/d9/17/a3b666f5ef9543cfd3c661d39d1e193abb9649d0cfbbfee3cf3b51d5af02/s3transfer-0.6.2-py3-none-any.whl.metadata
  Using cached s3transfer-0.6.2-py3-none-any.whl.metadata (1.8 kB)
  Obtaining dependency information for PyYAML<6.1,>=3.10 from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8cb42e0d285e/PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Us

In [3]:
import os
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

os.environ['BEDROCK_ASSUME_ROLE'] = role

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


arn:aws:iam::706553727873:role/service-role/AmazonSageMaker-ExecutionRole-20230713T143270


In [4]:
import json
import os
import sys

import boto3

module_path = "."
sys.path.append(os.path.abspath(module_path))

print(os.path.abspath(module_path))
from utils import bedrock, print_ww

/root/testing_customer_data


In [5]:
import json
import os
import sys

import boto3

module_path = "."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrockForYoga"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available model IDs include:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

In [6]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':2000, 'temperature':0})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

In [7]:
import traceback
import time
from typing import Any, Dict, List, Optional
class BedrockEmbeddingsCustom(BedrockEmbeddings):
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a Bedrock model.

        Args:
            texts: The list of texts to embed

        Returns:
            List of embeddings, one for each text.
        """
        print(f"BedrockEmbeddingsCustom: embed_docs():: lenght of texts={len(texts)}::")
        results = []
        counter = 1
        errors = []
        for text in texts:
            try:
                response = self._embedding_func(text)
                results.append(response)
                #print(f"BedrockEmbeddingsCustom: embed_docs()::processed doc_{counter}:")
                counter+=1
            except:
                print(f"BedrockEmbeddingsCustom: ERROR ={traceback.format_exc()}:: WAITING for 20 SEC")
                time.sleep(20) # 20 sec
                errors.append(text)
        
        print(f"BedrockEmbeddingsCustom: embed_docs(): TRYING Errors now:len={len(errors)}:")
        for text in errors:
            print(f"BedrockEmbeddingsCustom: embed_docs(): error :text={text}:")
            try:
                response = self._embedding_func(text)
                results.append(response)
                #print(f"BedrockEmbeddingsCustom: embed_docs()::processed doc_{counter}:")
                counter+=1
            except:
                print(f"BedrockEmbeddingsCustom: ERROR ={text}:: WAITING for 20 SEC")
                time.sleep(20) # 20 sec
                    
        return results
    
bedrock_embeddings = BedrockEmbeddingsCustom(client=boto3_bedrock)
bedrock_embeddings

BedrockEmbeddingsCustom(client=<botocore.client.Bedrock object at 0x7efd81935ae0>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-e1t-medium', model_kwargs=None, endpoint_url=None)

## Data Preparation
Let's first download some of the files to build our document store. For this example we will be using public IRS documents from [here](https://www.irs.gov/publications).

After downloading we can load the documents with the help of [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 512 tokens, which roughly translates to ~2000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [8]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./yoga_data/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [9]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 18 documents loaded is 2344 characters.
After the split we have 52 documents more than the original 18.
Average length among 52 documents (after split) is 844 characters.


We had 3 PDF documents which have been split into smaller ~500 chunks.

Now we can see how a sample embedding would look like for one of those chunks

In [10]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.47460938 -0.33789062 -0.50390625 ... -0.67578125  0.48242188
  0.24316406]
Size of the embedding:  (4096,)


Following the similar pattern embeddings could be generated for the entire corpus and stored in a vector store.

This can be easily done using [FAISS](https://github.com/facebookresearch/faiss) implementation inside [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html) which takes  input the embeddings model and the documents to create the entire vector store. Using the Index Wrapper we can abstract away most of the heavy lifting such as creating the prompt, getting embeddings of the query, sampling the relevant documents and calling the LLM. [VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation) helps us with that.

**⚠️⚠️⚠️ NOTE: it might take few minutes to run the following cell ⚠️⚠️⚠️**

In [11]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

vectorstore_faiss.save_local("faiss_index")

BedrockEmbeddingsCustom: embed_docs():: lenght of texts=52::
BedrockEmbeddingsCustom: embed_docs(): TRYING Errors now:len=0:


In [12]:
vectorstore_faiss = FAISS.load_local("faiss_index", bedrock_embeddings)
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

## Question Answering

Now that we have our vector store in place, we can start asking questions.

In [13]:
query = "Yoga for anxiety and back pain"

The first step would be to create an embedding of the query such that it could be compared with the documents

In [14]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

array([ 0.47851562, -0.5546875 , -0.46679688, ..., -0.671875  ,
        0.2890625 , -0.1796875 ])

We can use this embedding of the query to then fetch relevant documents.
Now our query is represented as embeddings we can do a similarity search of our query against our data store providing us with the most relevant information.

In [15]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: right Peloton Yoga class to fit your individual needs.
Find out more about Peloton’s on-demand yoga classes, including those specifically designed to
release stress.
Why it’s beneficial
If you’re dealing with back pain, yoga may be just what the doctor ordered. Yoga is a mind-body
therapy that’s often
recommended to treat not only back pain but the stress that accompanies it. The appropriate poses
can relax and strengthen
your body.
Practicing yoga for even a few minutes a day can help you gain more awareness of your body. This
will help you notice where
you’re holding tension and where you have imbalances. You can use this awareness to bring yourself
into balance and
alignment.
Keep reading to learn more about how these poses may be useful in treating back pain.
1. Cat-Cow
This gentle, accessible backbend stretches and mobilizes the spine. Practicing this pose also
stretches your torso, shoulders,
and neck.
M

Now we have the relevant documents, it's time to use the LLM to generate an answer based on these documents. 

We will take our inital prompt, together with our relevant documents which were retreived based on the results of our similarity search. We then by combining these create a prompt that we feed back to the model to get our result. At this point our model should give us highly informed information on how we can change the tire of our specific car as it was outlined in our manual.

LangChain provides an abstraction of how this can be done easily.

### Quick way
You have the possibility to use the wrapper provided by LangChain which wraps around the Vector Store and takes input the LLM.
This wrapper performs the following steps behind the scences:
- Takes input the question
- Create question embedding
- Fetch relevant documents
- Stuff the documents and the question into a prompt
- Invoke the model with the prompt and generate the answer in a human readable manner.

In [16]:
query = "Yoga for anxiety and back pain"
#query = "What are the different ways that the medication lithium can be administered?"
answer = wrapper_store_faiss.query(question=query, llm=llm)
print_ww(answer)

 Based on the information provided, yoga can be beneficial for both anxiety and back pain for the
following reasons:

- Yoga promotes relaxation and reduces stress hormones through physical postures, breathing
techniques, and mindfulness. This can help decrease anxiety levels and promote a greater sense of
calm.

- For back pain, yoga can relax and strengthen the body through gentle stretching and controlled
movements. Certain poses help release tension stored in the back and improve alignment.

- Practicing yoga increases body awareness which allows you to notice where you hold tension. This
awareness can then help bring your body into balance.

- Research shows yoga can provide small to moderate decreases in back pain intensity. One study
found similar improvements in chronic back pain from yoga and physical therapy over one year.

- The mindfulness aspect of yoga also encourages being present and letting go of worries, which can
reduce anxiety. Regular yoga practice improves overall

Let's ask a different question:

### Customisable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the helpf of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock, this particular model performs best if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [17]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: You will be acting as a Yoga Instructor.  Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "How do I reduce my daily stress from work doing yoga asanas?"
result = qa({"query": query})
print_ww(result['result'])

 Here are some tips to reduce work stress through yoga asanas:

- Practice standing forward folds like Uttanasana to release tension in the back and calm the
nervous system. Bend forward from your hips, letting your head and arms hang heavy.

- Do a few rounds of Cat-Cow stretches. As you arch and round your back with your breath, you'll
release tension in the spine.

- Try seated forward folds like Paschimottanasana. Sitting with legs extended, hinge at the hips to
fold forward over your legs. This stretches the hamstrings and back.

- Savasana, or Corpse Pose, is deeply relaxing. Lie on your back and let all tension melt away as
you focus on your breath. Stay for 5-10 minutes.

- Incorporate twisting poses like Ardha Matsyendrasana to stimulate your digestive system and
relieve tension in the back and shoulders.

- Try standing or seated wide-legged forward folds like Prasarita Padottanasana to open the hips and
hamstrings.

- End your practice with Sukhasana (Easy Pose) to promote t

#### -------------------------------------------------------------

### Now Try Titan models - PROMPT EXAMPLES

In [18]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

import json
import os
import sys

import boto3

module_path = "."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrockForYoga"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
)    
# - create the Titan  Model
llm_titan = Bedrock(model_id="amazon.titan-tg1-large", client=boto3_bedrock, model_kwargs={'maxTokenCount':200, 'temperature':0})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)
vectorstore_faiss_titan = FAISS.load_local("faiss_index", bedrock_embeddings)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


In [19]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: You will be acting as a Yoga Instructor.  Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm_titan,
    chain_type="stuff",
    retriever=vectorstore_faiss_titan.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "How can I cure both back pain and anxiety?"
result = qa({"query": query})
print_ww(result['result'])

Sorry - this model is designed to avoid profanity. Please see our content limitations page for more
information.


### Example 1: JSON prompting

JSON file up here is not completed, missed out the proposed indication section

## More optimal prompts for Titan - Try These!!

In [20]:
prompt_template = """You will be acting as a Yoga Instructor. Use the follow [instructions] to answer the [question] based on the [context] provided. Don't answer if the answer is not present in the [context]. Follow the [output_format] given below while responding.

context = {context} 

instructions = Use following instructions to answer the question above. 
Make sure to include following [attributes] in your answer as applicable.
- Teach one yoga pose for the {question} asked.
- Teach the benefit of the yoga pose for mental health
- Teach the benefit of the yoga pose for physical health


output_format = Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop



question = {question} 


output_format = Provide your output as a detailed paragraph that contains all [attributes] following all [instructions] above

answer: """


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm_titan,
    chain_type="stuff",
    retriever=vectorstore_faiss_titan.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "I am old and I have back pain in the lower back. How can I cure this using Yoga?"
result = qa({"query": query})
print_ww(result['result'])

Sorry - this model is designed to avoid profanity. Please see our content limitations page for more
information.


#### The above titan response is not fully completed. Let's try some more prompting techniques.

In [21]:
prompt_template = """You will be acting as a Yoga Instructor. Use the follow [instructions] to answer the [question] based on the [context] provided. Don't answer if the answer is not present in the [context]. Follow the [output_format] given below while responding.

context = {context} 

instructions = Use following instructions to answer the question above. 
Complete all the sentences and create a sense of comfort and understanding
Make sure to include following in your answer as applicable.
- Yoga Pose = Teach one yoga pose for the {question} asked.
- Mental Health Benefit = Teach the benefit of the yoga pose for mental health
- Physical Health Benefit = Teach the benefit of the yoga pose for physical health
Make sure to add all of these above in the [answer]


output_format = Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop
- All attributes are contained in the [answer]
- Give the [answer] with the emotion of care and concern for your students



question = {question} 


output_format = Provide your output as a detailed paragraph that contains all [attributes] following all [instructions] above
Make sure to think step by step and give a detailed response

answer: """


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm_titan,
    chain_type="stuff",
    retriever=vectorstore_faiss_titan.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = " How can I cure my neck pain using yoga?"
result = qa({"query": query})
print_ww(result['result'])

Sorry - this model is designed to avoid profanity. Please see our content limitations page for more
information.


#### This response that is given is incomplete. No matter how clear the input for the Titan model is, the output might or might not be complete in terms of the output

### Refer to the example below. Provide the instructions, and then give the context for better contextual answers, or Make the prompt template more detailed

In [22]:
prompt_template = """You will be acting as a Yoga Instructor. Use the follow [instructions] to answer the [question] based on the [context] provided. Don't answer if the answer is not present in the [context]. Follow the [output_format] given below while responding.

context = {context} 

question = {question} 

instructions = Use following instructions to answer the question above. 
- Do not hallucinate and only answer the question based on the context provided above. 
- Provide answer using the [output_format] at the end. 


instructions = Use following instructions to answer the question above. 
Complete all the sentences and create a sense of comfort and understanding
Make sure to include following in your answer as applicable.
- Yoga Pose = Teach one yoga pose for the {question} asked.
- Mental Health Benefit = Teach the benefit of the yoga pose for mental health
- Physical Health Benefit = Teach the benefit of the yoga pose for physical health
- Write an answer as a paragraph for a beginner student
- Write an answer as a paragraph for a intermediate student
- Write an answer as a paragraph for a advances student
Make sure to add all of these above in the [answer]


output_format = Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop
- All attributes are contained in the [answer]
- Give the [answer] with the emotion of care and concern for your students


output_format = Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop
- Add three paragraphs in your answer

output_format = Provide your output as a detailed paragraph that contains all [attributes] following all [instructions] above. 

End your answer with a completed sentence followed by a full stop

answer: """


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm_titan,
    chain_type="stuff",
    retriever=vectorstore_faiss_titan.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "What is the best yoga pose of a 24 year old woman stress and anxiety?"
result = qa({"query": query})
print_ww(result['result'])

The best yoga pose for a 24 year old woman to reduce stress and anxiety is the Child's Pose, also
known as Balasana. This pose involves kneeling on the floor and then sitting back on your heels,
lowering your torso between your thighs. Extend your arms forward or alongside your body and relax
your forehead on the mat or a block.

The Child's Pose is a simple yet effective pose that can help to calm the mind and reduce stress
levels. It is especially beneficial for those who experience anxiety, as it helps to soothe the
nervous system and promote feelings of safety and security. By taking a few minutes each day to
practice the Child's Pose, you can experience the benefits of yoga and reduce stress in your daily
life.

For a beginner student, it is important to start in a comfortable position and to focus on relaxing
the body. The student should imagine themselves as a child, safe and secure in their parent's arms,
and allow themselves to fully relax


Again, an incomplete response.

-------------------------------------------------------------------------------------------

### NOW, TRYING CLAUDE FOR OUT PROMPTS:

### Example 1: Simple answer first:

In [23]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: You will be acting as a Yoga Instructor. Use the follow <instructions></instructions> to answer the question based on the context provided. Don't answer if the answer is not present in the context. Follow the <output_format></output_format> given below while responding.

context = <context>{context}</context>

question = <question>{question}</question>

instructions = Use following instructions to answer the question above. 
- Do not hallucinate and only answer the question based on the context provided above. 

<instructions>
instructions = Use following instructions to answer the question above. 
Complete all the sentences and create a sense of comfort and understanding
Make sure to include following in your answer as applicable.
- Yoga Pose = Teach one yoga pose for the {question} asked.
- Mental Health Benefit = Teach the benefit of the yoga pose for mental health
- Physical Health Benefit = Teach the benefit of the yoga pose for physical health
- Write an answer as a paragraph for a beginner student
- Write an answer as a paragraph for a intermediate student
- Write an answer as a paragraph for a advances student
Make sure to add all of these above in the [answer]
</instructions>

<output_format>
output_format = Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop
- All attributes are contained in the answer
- Give the answer with the emotion of care and concern for your students
</output_format>


Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop
- Add three paragraphs in your answer

Provide your output as a detailed paragraph that contains all attributes following all instructions above. 
answer: """

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "What is the best yoga pose of a 24 year old woman stress and anxiety?"
result = qa({"query": query})
print_ww(result['result'])

 <answer>
Here is a calming yoga pose that can help relieve stress and anxiety for a 24 year old woman:

For the beginner student:
Child's Pose (Balasana) is an excellent stress-relieving yoga pose. To do Child's Pose, kneel on the
floor and sit back on your heels. Then lower your torso between your thighs and extend your arms in
front of you or relax them by your sides. Rest your forehead on the floor or a block. This pose
gently stretches the hips, thighs, ankles and back which helps release tension. It also calms the
mind and nervous system, providing a sense of comfort and security. Hold Child's Pose for a few
long, deep breaths. It can be held for up to a few minutes for full relaxation. Regular practice of
Child's Pose can alleviate anxiety and promote feelings of calmness.

For the intermediate student:
A calming yoga pose to help relieve stress and anxiety is Legs Up the Wall Pose (Viparita Karani).
To do this pose, lie on your back near a wall. Extend your legs straight up aga

### Example 2: Give a step by step approach:

In [24]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: You will be acting as a Yoga Instructor. Use the follow <instructions></instructions> to answer the question based on the context provided. Don't answer if the answer is not present in the context. Follow the <output_format></output_format> given below while responding.

context = <context>{context}</context>

question = <question>{question}</question>

instructions = Use following instructions to answer the question above. 
- Do not hallucinate and only answer the question based on the context provided above. 

<instructions>
instructions = Use following instructions to answer the question above. 
Complete all the sentences and create a sense of comfort and understanding
Make sure to include following in your answer as applicable.
- Yoga Pose = Teach one yoga pose for the {question} asked.
- Mental Health Benefit = Teach the benefit of the yoga pose for mental health
- Physical Health Benefit = Teach the benefit of the yoga pose for physical health
- Write an answer as a paragraph for a beginner student
- Write an answer as a paragraph for a intermediate student
- Write an answer as a paragraph for a advances student
Make sure to add all of these above in the [answer]
</instructions>

<output_format>
output_format = Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop
- All attributes are contained in the answer
- Give the answer with the emotion of care and concern for your students
</output_format>


Provide your output as a text based paragraph that follows the instructions below
- Each and every sentence is complete, ending with a full stop

Assistant: Can I think step by step?

Human: Yes, you can think step by step. 

Assistant: Is there anything else I can add in my answer?

Human: Yes, add elements of care and concern. Also give steps for both hindi and english readers.

answer: """

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "What is the best yoga pose of a 24 year old woman stress and anxiety?"
result = qa({"query": query})
print_ww(result['result'])

 Here is a comprehensive answer for a 24 year old woman looking to relieve stress and anxiety
through yoga:

For a beginner student:
Namaste. I understand you are looking for a yoga pose to help relieve stress and anxiety. Child's
pose, or Balasana, is an excellent beginner pose that promotes deep relaxation. To do this pose,
start on your hands and knees. Then sit back on your heels and lower your torso between your thighs.
Reach your arms forward and relax your forehead down. Breathe deeply and focus on releasing any
tension you feel. Hold for a few breaths. Balasana gently stretches your back, shoulders and neck to
release tension. It also calms the mind and nervous system. Practicing this nurturing pose will
provide comfort, security and tranquility.

For an intermediate student:
Namaste. As an intermediate student, I would recommend Legs up the Wall pose, or Viparita Karani, to
relieve stress and anxiety. To do this pose, lie on your back near a wall. Extend your legs straight
up 

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

# Define the SageMaker estimator
role = role
image_uri = "your-bedrock-container-image"
estimator = Estimator(
    role=role,
    image_uri=image_uri,
    instance_count=1,
    instance_type="ml.p3.2xlarge",  # Choose an appropriate instance type
    base_job_name="your-bedrock-training-job",
    hyperparameters={
        "your_hyperparameter": "value",
        # Add other hyperparameters here
    },
)

# Set the training data location
train_data = "s3://your-s3-bucket/path/to/train/data"
estimator.fit({"train": train_data})
